In [1]:
import sys
print(sys.version)
import tensorflow
import os
import glob
import cv2
from pathlib import Path
from tensorflow import keras
import numpy as np
import pandas as pd
import shutil
import struct
from tensorflow.keras.layers import Input
#import keras
#keras.__version__

3.8.5 (default, May 27 2021, 13:30:53) 
[GCC 9.3.0]


## Preparing rickshaw dataset
Preparing rickshaw dataset according to Darknet requirements

In [20]:
# For .py file use the following code to get file path
# print(Path(__file__).resolve().parent)
notebook_path = Path(os.path.abspath(''))
root_dir = notebook_path.parent
rickshaw_labels = notebook_path.joinpath("images", "train", "train_labels.csv")
print(notebook_path)
print(root_dir)
data_path = root_dir /"darknet" / "build" / "darknet" / "x64" / "data"
obj_path = data_path / "obj"


/home/suleman/Fake_directories/darknet
/home/suleman/Fake_directories


In [3]:
src_dir = notebook_path / "rikshaw_dataset"
for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
    shutil.copy(jpgfile, obj_path)

### Delete already existing file
To delete the already existing files from the darknet directory so that the same annotation is not rewritten in the file, thus avoiding conflict

In [4]:
print(obj_path)
for file in glob.glob(str(obj_path) + '/*.txt'):
    print(file)
    if os.path.exists(file):
      os.remove(file)
# ./darknet/build/darknet/x64/data/
# print(rickshaw_labels)

/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000499749.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000507927.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000221441.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000393290.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000368833.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000196766.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000000113.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000049283.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000188575.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000524333.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000033068.txt
/home/suleman/Fake_di

/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000376972.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000262275.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000549048.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000328185.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000393724.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000155811.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000401526.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000065705.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000459000.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000196811.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000295015.txt
/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/000000221394.txt
/hom

### Reading file of rickshaw csv

In [5]:
df_annot = pd.read_csv(rickshaw_labels)
df_annot

,filename,width,height,class,xmin,ymin,xmax,ymax
0,001_auto-rickshaw.jpg,1002,564,auto,229,92,777,455
1,001_Electric-Auto-Rickshaw.jpg,500,500,auto,255,75,482,384
2,001_Electric-Auto-Rickshaw.jpg,500,500,auto,17,95,268,359
3,001_hqdefault.jpg,480,360,auto,113,64,394,282
4,002_auto-rickshaw.jpg,600,397,auto,12,39,238,350
...,...,...,...,...,...,...,...,...
140,The_New_Auto_Rickshaw_Owners_Drivers_Associati...,350,300,auto,239,154,323,272
141,_3d066786-6a35-11e7-95fb-ec6334583ea6.JPG,960,669,auto,6,11,950,669
142,A.jpg,773,571,auto,36,43,739,541
143,autorickshaw.53.jpg,1280,791,auto,347,22,1170,730


In [6]:
df_annot["class"].unique()

array(['auto'], dtype=object)

### Finding cx, cy, w, and h
Finding center-x, center-y coordinates and width and height from x-min, y-min, x-max and y-max

In [7]:
df_annot["cx"] = (df_annot["xmin"] + df_annot["xmax"])/2
df_annot["cy"] = (df_annot["ymin"] + df_annot["ymax"])/2
df_annot["w"] = df_annot["xmax"] - df_annot["xmin"]
df_annot["h"] = df_annot["ymax"] - df_annot["ymin"]
df_annot["class"] = 0
df_annot

,filename,width,height,class,xmin,ymin,xmax,ymax,cx,cy,w,h
0,001_auto-rickshaw.jpg,1002,564,0,229,92,777,455,503.0,273.5,548,363
1,001_Electric-Auto-Rickshaw.jpg,500,500,0,255,75,482,384,368.5,229.5,227,309
2,001_Electric-Auto-Rickshaw.jpg,500,500,0,17,95,268,359,142.5,227.0,251,264
3,001_hqdefault.jpg,480,360,0,113,64,394,282,253.5,173.0,281,218
4,002_auto-rickshaw.jpg,600,397,0,12,39,238,350,125.0,194.5,226,311
...,...,...,...,...,...,...,...,...,...,...,...,...
140,The_New_Auto_Rickshaw_Owners_Drivers_Associati...,350,300,0,239,154,323,272,281.0,213.0,84,118
141,_3d066786-6a35-11e7-95fb-ec6334583ea6.JPG,960,669,0,6,11,950,669,478.0,340.0,944,658
142,A.jpg,773,571,0,36,43,739,541,387.5,292.0,703,498
143,autorickshaw.53.jpg,1280,791,0,347,22,1170,730,758.5,376.0,823,708


In [8]:
len(df_annot["filename"].unique())

102

Scaling center-x, center-y coordinates and width with respect to image resolution

In [9]:
df_annot["cx"] = (df_annot["cx"]/df_annot["width"]).round(6)
df_annot["w"] = (df_annot["w"]/df_annot["width"]).round(6)
df_annot["cy"] = (df_annot["cy"]/df_annot["height"]).round(6)
df_annot["h"] = (df_annot["h"]/df_annot["height"]).round(6)
df_annot = df_annot.applymap(str)
df_annot

,filename,width,height,class,xmin,ymin,xmax,ymax,cx,cy,w,h
0,001_auto-rickshaw.jpg,1002,564,0,229,92,777,455,0.501996,0.484929,0.546906,0.643617
1,001_Electric-Auto-Rickshaw.jpg,500,500,0,255,75,482,384,0.737,0.459,0.454,0.618
2,001_Electric-Auto-Rickshaw.jpg,500,500,0,17,95,268,359,0.285,0.454,0.502,0.528
3,001_hqdefault.jpg,480,360,0,113,64,394,282,0.528125,0.480556,0.585417,0.605556
4,002_auto-rickshaw.jpg,600,397,0,12,39,238,350,0.208333,0.489924,0.376667,0.783375
...,...,...,...,...,...,...,...,...,...,...,...,...
140,The_New_Auto_Rickshaw_Owners_Drivers_Associati...,350,300,0,239,154,323,272,0.802857,0.71,0.24,0.393333
141,_3d066786-6a35-11e7-95fb-ec6334583ea6.JPG,960,669,0,6,11,950,669,0.497917,0.508221,0.983333,0.983558
142,A.jpg,773,571,0,36,43,739,541,0.501294,0.511384,0.909444,0.872154
143,autorickshaw.53.jpg,1280,791,0,347,22,1170,730,0.592578,0.475348,0.642969,0.89507


### Writing annotation to text file
One text file for every image, every text file name is after the name of orignal image but .txt instead of .jpg, each text file contains annotation(s) of bounding box

In [10]:
df = df_annot.set_index("filename")

for index, row in df.iterrows():
    file_path = obj_path / (index[:-3] + "txt")
    print(file_path)
#     f= open(file_path, "a+")
    string = row["class"]+ ' '+ row["cx"]+ ' '+row["cy"]+ ' '+row["w"]+ ' '+row["h"] + '\n'
    print(string)
#     f.write(string)
#     f.close()

/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/001_auto-rickshaw.txt
0 0.501996 0.484929 0.546906 0.643617

/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/001_Electric-Auto-Rickshaw.txt
0 0.737 0.459 0.454 0.618

/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/001_Electric-Auto-Rickshaw.txt
0 0.285 0.454 0.502 0.528

/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/001_hqdefault.txt
0 0.528125 0.480556 0.585417 0.605556

/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/002_auto-rickshaw.txt
0 0.208333 0.489924 0.376667 0.783375

/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/002_auto-rickshaw.txt
0 0.634167 0.552897 0.661667 0.894207

/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/002_hqdefault.txt
0 0.479167 0.522222 0.920833 0.727778

/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/003_hqdefault.txt
0 0.507292 0.519444 0.23125 0.483333

/ho

0 0.496795 0.514925 0.858974 0.970149

/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/images_q=tbn_ANd9GcQDNKwtnEP1sCvW_zsQ9PbAUrICgiY_KEGbD-CxXg3Ax_kyt-4I.txt
0 0.44697 0.582944 0.745455 0.829439

/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/images_q=tbn_ANd9GcQE4uQPgUJ0-z1DDZ1aSTILljYVNZ4GgZTB8o8zCpw7O5Pw9-h0.txt
0 0.532051 0.514354 0.833333 0.818182

/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/images_q=tbn_ANd9GcQJaNSwk9K0wpw0SNXxL2RISkBeM2c3T4fj870yofkxmTfDvfAE.txt
0 0.153125 0.601042 0.303125 0.797917

/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/images_q=tbn_ANd9GcQJaNSwk9K0wpw0SNXxL2RISkBeM2c3T4fj870yofkxmTfDvfAE.txt
0 0.514062 0.626042 0.425 0.747917

/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/images_q=tbn_ANd9GcQJaNSwk9K0wpw0SNXxL2RISkBeM2c3T4fj870yofkxmTfDvfAE.txt
0 0.85625 0.551042 0.2875 0.63125

/home/suleman/Fake_directories/darknet/build/darknet/x64/data/obj/images_q

### Creating train.txt
Creating (if not present) a txt file, which contatins name and path of every tarining image

In [11]:
# Remove file/diectory if alreay exists
#./build/darknet/x64/data/obj/
for name in df_annot["filename"].unique():
    traintxt_path = data_path / "train.txt"
    print(name)
    t= open(traintxt_path, "a+")
#     t.write("./build/darknet/x64/data/obj/" + name + '\n')
    t.close()

001_auto-rickshaw.jpg
001_Electric-Auto-Rickshaw.jpg
001_hqdefault.jpg
002_auto-rickshaw.jpg
002_hqdefault.jpg
003_hqdefault.jpg
004_Auto-Rickshaw.jpg
07 Auto rickshaw.jpg
1200px-AutoRickshaw.jpg
135574424.jpg
136821422.jpg
1497072140-auto.JPG
18BG_AUTO1-570x356.jpg
20121204_102840.jpg
20auto-rickshaw2.jpg
240px-Autorickshaw_on_Raj_Path_New_Delhi.JPG
240px-Goa_Rickshaw.jpg
240px-More_Karachi_Main.jpg
240px-RickshawPak.jpg
24171888592_8d9e387111_b.jpg
464740585.jpg
52.jpg
800px_COLOURBOX11647521.jpg
885937694.jpg
891270888.jpg
accident-story-fb_647_051716124359.jpg
add-innovation-500x500.jpg
an-auto-rickshaw-passes-a-sign-on-a-wall-in-allepuzha-alleppey-kerala-fbaa7p.jpg
API-Garv-RE-Passenger-Auto-Rickshaw.jpg
Auto-094.jpg
auto-refusal_c981c4d4-dd91-11e6-a538-54bd197a5a1b.jpg
Auto-Rickshaw-3-Seater-4-810x734.jpg
auto-rickshaw-640x374.jpg
auto-rickshaw-center-khamla-nagpur-tempo-travellers-on-hire-3kbzzn.jpg
auto-rickshaw-challenge.jpg
Auto-Rickshaw-Driver-Travels-1-Lakh-Kilometer-With-H

In [12]:
# !pip install pycocotools

In [13]:
# !git clone https://github.com/cocodataset/cocoapi

## Building our COCO dataset
Selecting and Downloaded images which belong to person, bicycle, car, motorcycle, bus and truck class. The `instances_train2017.json` was used to extract the IDs of the images which contains our required object (classes). Images were downloaded using those IDs and their URL


In [14]:
coco_annot_path = notebook_path /"cocoapi" / "annotations" / "instances_train2017.json"

In [15]:
from pycocotools.coco import COCO
import requests

coco = COCO(coco_annot_path)
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))
classes = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck']
catIds = coco.getCatIds(catNms=classes)
print(catIds)

count = 0
for c in catIds:
    if count>0:
        imgIds2= coco.getImgIds(catIds=c)[:125]
        imgIds+=imgIds2
    else:
        imgIds=coco.getImgIds(catIds=c)[:125]
    count+=1
    # print(imgIds)
    print(len(imgIds))
images = coco.loadImgs(imgIds)
print("imgIds: ", imgIds)
print("images: ", images)

for im in images:
    print("im: ", im)
    img_data = requests.get(im['coco_url']).content
################# Change path to download images directly to darknet###########################
    with open(obj_path / im['file_name'], 'wb') as handler:
        handler.write(img_data)

loading annotations into memory...
Done (t=13.36s)
creating index...
index created!
COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

[1, 2, 3, 4, 6, 8]
125
250
375
500
625
750
imgIds:  [262145, 262146, 524291, 262148, 393223, 393224, 524297, 393227, 131084, 393230, 262161, 131089, 524311, 393241, 524314, 393243, 262171, 131101, 524317, 262175, 524320, 393251, 131108, 524325, 36, 131115, 524333, 262191, 49,

im:  {'license': 1, 'file_name': '000000262146.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000262146.jpg', 'height': 640, 'width': 480, 'date_captured': '2013-11-19 23:07:16', 'flickr_url': 'http://farm6.staticflickr.com/5090/5341741494_1f653cdb80_z.jpg', 'id': 262146}
im:  {'license': 3, 'file_name': '000000524291.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000524291.jpg', 'height': 426, 'width': 640, 'date_captured': '2013-11-18 09:59:07', 'flickr_url': 'http://farm2.staticflickr.com/1045/934293170_d1b2cc58ff_z.jpg', 'id': 524291}
im:  {'license': 1, 'file_name': '000000262148.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000262148.jpg', 'height': 512, 'width': 640, 'date_captured': '2013-11-20 05:50:03', 'flickr_url': 'http://farm5.staticflickr.com/4028/4549977479_547e6b22ae_z.jpg', 'id': 262148}
im:  {'license': 3, 'file_name': '000000393223.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000393223.jpg', 'height': 48

im:  {'license': 4, 'file_name': '000000393267.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000393267.jpg', 'height': 640, 'width': 425, 'date_captured': '2013-11-25 20:40:17', 'flickr_url': 'http://farm4.staticflickr.com/3154/2973532161_d07d1deb24_z.jpg', 'id': 393267}
im:  {'license': 3, 'file_name': '000000393268.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000393268.jpg', 'height': 480, 'width': 640, 'date_captured': '2013-11-21 19:51:33', 'flickr_url': 'http://farm3.staticflickr.com/2249/2238886225_0c5fee68b4_z.jpg', 'id': 393268}
im:  {'license': 3, 'file_name': '000000262197.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000262197.jpg', 'height': 428, 'width': 640, 'date_captured': '2013-11-24 01:12:40', 'flickr_url': 'http://farm2.staticflickr.com/1085/739335222_ba38e974fb_z.jpg', 'id': 262197}
im:  {'license': 3, 'file_name': '000000393271.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000393271.jpg', 'height': 48

im:  {'license': 3, 'file_name': '000000524401.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000524401.jpg', 'height': 427, 'width': 640, 'date_captured': '2013-11-22 21:18:38', 'flickr_url': 'http://farm4.staticflickr.com/3527/3714366780_ebf3c1f74a_z.jpg', 'id': 524401}
im:  {'license': 2, 'file_name': '000000000113.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000000113.jpg', 'height': 640, 'width': 416, 'date_captured': '2013-11-15 18:15:09', 'flickr_url': 'http://farm8.staticflickr.com/7068/6809404644_28dee247c9_z.jpg', 'id': 113}
im:  {'license': 2, 'file_name': '000000262260.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000262260.jpg', 'height': 427, 'width': 640, 'date_captured': '2013-11-16 14:22:56', 'flickr_url': 'http://farm3.staticflickr.com/2554/3930333165_2c090fcf8e_z.jpg', 'id': 262260}
im:  {'license': 4, 'file_name': '000000393333.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000393333.jpg', 'height': 387,

im:  {'license': 4, 'file_name': '000000131245.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000131245.jpg', 'height': 417, 'width': 640, 'date_captured': '2013-11-14 21:08:10', 'flickr_url': 'http://farm5.staticflickr.com/4094/4924917735_62a9958d8a_z.jpg', 'id': 131245}
im:  {'license': 3, 'file_name': '000000524467.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000524467.jpg', 'height': 426, 'width': 640, 'date_captured': '2013-11-21 05:43:13', 'flickr_url': 'http://farm2.staticflickr.com/1089/4599967769_99067566f7_z.jpg', 'id': 524467}
im:  {'license': 1, 'file_name': '000000524470.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000524470.jpg', 'height': 640, 'width': 640, 'date_captured': '2013-11-15 01:33:49', 'flickr_url': 'http://farm5.staticflickr.com/4124/5198007317_7810e285e5_z.jpg', 'id': 524470}
im:  {'license': 1, 'file_name': '000000524476.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000524476.jpg', 'height': 4

...

im:  {'license': 1, 'file_name': '000000278656.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000278656.jpg', 'height': 375, 'width': 500, 'date_captured': '2013-11-24 07:28:14', 'flickr_url': 'http://farm2.staticflickr.com/1170/664254030_2a5a6fabec_z.jpg', 'id': 278656}
im:  {'license': 3, 'file_name': '000000221311.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000221311.jpg', 'height': 425, 'width': 640, 'date_captured': '2013-11-18 08:35:17', 'flickr_url': 'http://farm7.staticflickr.com/6207/6105754811_4e29e7340d_z.jpg', 'id': 221311}
im:  {'license': 3, 'file_name': '000000229507.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000229507.jpg', 'height': 375, 'width': 500, 'date_captured': '2013-11-16 22:25:43', 'flickr_url': 'http://farm2.staticflickr.com/1035/1067835644_beec84fe85_z.jpg', 'id': 229507}
im:  {'license': 1, 'file_name': '000000524420.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000524420.jpg', 'height': 46

im:  {'license': 3, 'file_name': '000000434380.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000434380.jpg', 'height': 427, 'width': 640, 'date_captured': '2013-11-20 06:18:58', 'flickr_url': 'http://farm5.staticflickr.com/4022/4381782042_fbeaa8b7af_z.jpg', 'id': 434380}
im:  {'license': 3, 'file_name': '000000311501.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000311501.jpg', 'height': 640, 'width': 426, 'date_captured': '2013-11-16 12:13:02', 'flickr_url': 'http://farm8.staticflickr.com/7054/6983231161_301e7b5cb6_z.jpg', 'id': 311501}
im:  {'license': 3, 'file_name': '000000024779.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000024779.jpg', 'height': 640, 'width': 480, 'date_captured': '2013-11-16 21:26:55', 'flickr_url': 'http://farm5.staticflickr.com/4006/4517190748_13057e68f6_z.jpg', 'id': 24779}
im:  {'license': 3, 'file_name': '000000344271.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000344271.jpg', 'height': 48

im:  {'license': 1, 'file_name': '000000516344.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000516344.jpg', 'height': 375, 'width': 500, 'date_captured': '2013-11-17 17:06:55', 'flickr_url': 'http://farm3.staticflickr.com/2399/2210469536_c37a1bbf9a_z.jpg', 'id': 516344}
im:  {'license': 3, 'file_name': '000000221433.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000221433.jpg', 'height': 346, 'width': 500, 'date_captured': '2013-11-20 18:29:34', 'flickr_url': 'http://farm2.staticflickr.com/1182/613431344_7b813f27cd_z.jpg', 'id': 221433}
im:  {'license': 6, 'file_name': '000000532735.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000532735.jpg', 'height': 375, 'width': 500, 'date_captured': '2013-11-15 15:42:40', 'flickr_url': 'http://farm1.staticflickr.com/32/56901601_b1b65281f9_z.jpg', 'id': 532735}
im:  {'license': 2, 'file_name': '000000557313.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000557313.jpg', 'height': 480, '

im:  {'license': 5, 'file_name': '000000065567.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000065567.jpg', 'height': 480, 'width': 640, 'date_captured': '2013-11-17 00:38:49', 'flickr_url': 'http://farm4.staticflickr.com/3598/3443307025_0437e94714_z.jpg', 'id': 65567}
im:  {'license': 4, 'file_name': '000000360480.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000360480.jpg', 'height': 428, 'width': 640, 'date_captured': '2013-11-21 03:38:06', 'flickr_url': 'http://farm8.staticflickr.com/7051/6958133637_63e330d8f0_z.jpg', 'id': 360480}
im:  {'license': 1, 'file_name': '000000196639.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000196639.jpg', 'height': 500, 'width': 375, 'date_captured': '2013-11-21 01:42:39', 'flickr_url': 'http://farm3.staticflickr.com/2228/2054616749_ca42d8bfe6_z.jpg', 'id': 196639}
im:  {'license': 3, 'file_name': '000000327713.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000327713.jpg', 'height': 42

KeyboardInterrupt: 

In [ ]:
print(len(images))

### Converting the annotation according to Darknet format
Annotations of the images were converted into the required formate of Darknet.

A txt file for every image named after the name of image (replaced .jpg with .txt). It coontains annotations of bounding box.

A txt file which contatins names of all of the images with thier path

In [ ]:
classes = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck']
# with open('annotations_download_' +'.csv', mode='w', newline='') as annot:
for im in images:
    annIds = coco.getAnnIds(imgIds=im['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    file_name = im['file_name']
    file_path = obj_path / (file_name[:-3] + "txt")
    print(file_path)
    string = "./build/darknet/x64/data/obj/" + file_name
    t = open(traintxt_path, "r")
    if string in t.read():
        continue
    t.close()
    t = open(traintxt_path, "a+")
    t.write(string + '\n')
    t.close()
    
    for i in range(len(anns)):
        image_w = im['width']
        image_h = im['height']
        w = round(anns[i]['bbox'][2]/image_w, 6)
        h = round(anns[i]['bbox'][3]/image_h, 6)
        cx = round((anns[i]['bbox'][0] + w/2)/image_w, 6)
        cy = round((anns[i]['bbox'][1] + h/2)/image_h, 6)
        cat = anns[i]['category_id']
        if(anns[i]['category_id']==8):
            cat = 5
        string = str(cat) + ' '+ str(cx)+ ' '+str(cy)+ ' '+str(w)+ ' '+str(h) + '\n'
        if os.path.isfile(file_path):
            f = open(file_path, "r")
            if string in f.read():
                continue
            f.close()
        f= open(file_path, "a+")
        f.write(string)
        f.close()


In [ ]:
anns[0]

In [ ]:
# !pip install scikit-image

In [ ]:
import matplotlib.pyplot as plt
import skimage.io as io
img = images[0]
I = io.imread(img['coco_url'])
plt.axis('off')
plt.imshow(I)
plt.show()


In [ ]:


# load and display instance annotations
plt.imshow(I); plt.axis('off')
annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
anns = coco.loadAnns(annIds)
coco.showAnns(anns)



In [ ]:
anns = coco.loadAnns(annIds)
for i in anns:
    [x,y,w,h] = i['bbox']
    cv2.rectangle(I, (int(x), int(y)), (int(x+w), int(y+h)), (255,0,0), 5)
    cv2.imshow(' ',I)
plt.show()

In [32]:
import subprocess
darknet_path = root_dir / "darknet"
os.chdir(darknet_path)
print(os.getcwd())
subprocess.run("./darknet detector train ./build/darknet/x64/data/obj.data ./cfg/yolo-obj.cfg ./build/darknet/x64/yolov4.conv.137", capture_output=True)

/home/suleman/Fake_directories/darknet


CompletedProcess(args='ls', returncode=0, stdout=b'build\n', stderr=b'')

In [1]:

import wget

url = "https://drive.google.com/uc?export=download&id=1Cg-hbtlYdMiMFZ88owFIRSfgpE9WI0av"
wget(url)

ModuleNotFoundError: No module named 'wget'